In [1]:
# Import Dependencies
import requests
import json
import os
import csv
from pathlib import Path
from pandas import DataFrame
from pandas.io.json import json_normalize
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [2]:
url = "https://data.lacity.org/resource/s49e-q6j2.json?$limit=33989"

response = requests.get(url)
la_meter_inventory = json.loads(response.content)

# This will be a dictionary of the JSON response
la_meter_inventory

[{'spaceid': 'VS336',
  'blockface': '4900 SUNSET BL',
  'metertype': 'Single-Space',
  'ratetype': 'FLAT',
  'raterange': '$1.00',
  'timelimit': '2HR',
  'parkingpolicy': 'TANS 4P-7P Mon-Fri | PKG 8A-4P Mon-Fri, 8A-8P Sat',
  'latlng': {'latitude': '34.098024', 'longitude': '-118.296664'},
  ':@computed_region_qz3q_ghft': '23445',
  ':@computed_region_k96s_3jcv': '430',
  ':@computed_region_tatf_ua23': '456',
  ':@computed_region_ur2y_g4cx': '16',
  ':@computed_region_kqwf_mjcx': '8',
  ':@computed_region_2dna_qi2s': '58'},
 {'spaceid': 'ED396',
  'blockface': '700 E PICO BLVD',
  'metertype': 'Single-Space',
  'ratetype': 'TOD',
  'raterange': '$0.5 - $1',
  'timelimit': '2HR',
  'parkingpolicy': '8A-6P Mon-Sat',
  'latlng': {'latitude': '34.032054', 'longitude': '-118.252521'},
  ':@computed_region_qz3q_ghft': '23082',
  ':@computed_region_k96s_3jcv': '703',
  ':@computed_region_tatf_ua23': '1018',
  ':@computed_region_kqwf_mjcx': '9',
  ':@computed_region_2dna_qi2s': '76'},
 {'spa

In [3]:
la_meter_inventory = json.dumps(la_meter_inventory, indent=4, sort_keys=True)

In [4]:
writeFile = open('la_meter_inventory.json', 'w')
writeFile.write(la_meter_inventory)
writeFile.close()

In [5]:
# read json into dataframe
json_file = "la_meter_inventory.json"
la_meter_inventory_df = pd.read_json(json_file)
la_meter_inventory_df

,:@computed_region_2dna_qi2s,:@computed_region_k96s_3jcv,:@computed_region_kqwf_mjcx,:@computed_region_qz3q_ghft,:@computed_region_tatf_ua23,:@computed_region_ur2y_g4cx,blockface,latlng,metertype,parkingpolicy,raterange,ratetype,spaceid,stcl,timelimit
0,58.0,430.0,8.0,23445,456.0,16.0,4900 SUNSET BL,"{'latitude': '34.098024', 'longitude': '-118.2...",Single-Space,"TANS 4P-7P Mon-Fri | PKG 8A-4P Mon-Fri, 8A-8P Sat",$1.00,FLAT,VS336,NaN,2HR
1,76.0,703.0,9.0,23082,1018.0,NaN,700 E PICO BLVD,"{'latitude': '34.032054', 'longitude': '-118.2...",Single-Space,8A-6P Mon-Sat,$0.5 - $1,TOD,ED396,NaN,2HR
2,76.0,693.0,9.0,23076,1291.0,NaN,1300 S GRAND AVE,"{'latitude': '34.038125', 'longitude': '-118.2...",Single-Space,8A-8P Mon-Sat,$0.50,FLAT,CB211A,NaN,4HR
3,90.0,473.0,8.0,23444,1297.0,NaN,3100 SUNSET BL,"{'latitude': '34.084343', 'longitude': '-118.2...",Single-Space,8A-8P Mon-Sat,$1.00,FLAT,SL408,NaN,2HR
4,18.0,229.0,3.0,19733,288.0,NaN,7001 VAN NUYS BL,"{'latitude': '34.198035', 'longitude': '-118.4...",Single-Space,8A-8P Mon-Sat,$1.00,FLAT,VN1,NaN,2HR
5,82.0,414.0,8.0,23446,1303.0,NaN,1700 EL CERRITO PL,"{'latitude': '34.102349', 'longitude': '-118.3...",Single-Space,"TANS 8A-5A Fri | PKG 8A-8P Mon-Thu, 8A-12A Fri...",$1.5 - $3,TOD,HO1021A,12P-3P Mon,2HR
6,68.0,571.0,11.0,23078,628.0,NaN,800 BEACON AV,"{'latitude': '34.051792', 'longitude': '-118.2...",Single-Space,8A-6P Mon-Sat,$1.00,FLAT,WU759,12P-3P Thu,2HR
7,76.0,693.0,9.0,23076,1291.0,NaN,1301 S HOPE ST,"{'latitude': '34.039121', 'longitude': '-118.2...",Single-Space,8A-8P Mon-Sat,$1 - $2,TOD,CB118,NaN,4HR
8,26.0,629.0,6.0,23679,1095.0,NaN,8201 3RD ST,"{'latitude': '34.072672', 'longitude': '-118.3...",Single-Space,"8A-8P Mon-Sat, 11A-8P Sun",$1.00,FLAT,WF60,NaN,2HR
9,72.0,855.0,6.0,23443,748.0,9.0,1600 PONTIUS AV,"{'latitude': '34.04847', 'longitude': '-118.44...",Single-Space,8A-8P Mon-Sat,$1.00,FLAT,WS490,NaN,1HR


start db connection here

In [2]:
# Create database connection
connection_string = "root:root@127.0.0.1:3306/parking_db"
engine = create_engine(f'mysql+pymysql://{connection_string}')

In [3]:
# Confirm tables
engine.table_names()

['meters_inventory']

In [ ]:
# Load DataFrames into database
plane_attributes_df.to_sql(name = 'meters_inventory', con = engine, if_exists = 'append', index = True) 

In [5]:
def transform_record(r):
    """
    Define the dictionary that I want to store.
    """
    # Remove the leading $, convert to float
#     raterange = float(r['raterange'][1:])
#     drop([':@computed_region_2dna_qi2s',
#                             ':@computed_region_k96s_3jcv',
#                             ':@computed_region_kqwf_mjcx',
#                             ':@computed_region_qz3q_ghft',
#                             ':@computed_region_tatf_ua23', 
#                             ':@computed_region_ur2y_g4cx'], 
#                            axis = 1, inplace = True)
    return {
        "myspaceid": r['spaceid'],
        "latitude": float(r['latlng']['latitude']),
        "longitude": float(r['latlng']['longitude']),
#         "raterange": raterange,
    }


In [6]:
def store_record(r):
    insert_statement = """INSERT INTO meters_inventory (`spaceid`, `latitude`, `longitude`) VALUES ("%s", "%s", "%s")""" % (r['myspaceid'], r['latitude'], r['longitude'])
    # insert into database

In [2]:
api_url = 'https://data.lacity.org/resource/s49e-q6j2.json'
limit_number = 0
offset_number = 0

for x in range(0, 4):
    url = api_url + '?$limit=' + str(limit_number + 10000) + '&$offset=' + str(offset_number)
    response = json.dumps(requests.get(url).json())
    writeFile = open(f"page_{x}.json", 'w')
    writeFile.write(response)
    writeFile.close()
    print("API call "f"{x}"" completed")
    offset_number += 10000

API call 0 completed
API call 1 completed
API call 2 completed
API call 3 completed


In [7]:
# read json into dataframe
for x in range(0, 4):
    filename = f"page_{x}.json"
    with open(filename) as json_file:
        results = json.load(json_file)
        for r in results:
            r = transform_record(r)
#             store_record(r)
            print(r)
#             
# json_file = "page_30000.json"
# la_meter_inventory_df = pd.read_json(json_file)
# la_meter_inventory_df

{'myspaceid': 'VS336', 'latitude': 34.098024, 'longitude': -118.296664}
{'myspaceid': 'ED396', 'latitude': 34.032054, 'longitude': -118.252521}
{'myspaceid': 'CB211A', 'latitude': 34.038125, 'longitude': -118.264675}
{'myspaceid': 'SL408', 'latitude': 34.084343, 'longitude': -118.274463}
{'myspaceid': 'VN1', 'latitude': 34.198035, 'longitude': -118.44891}
{'myspaceid': 'HO1021A', 'latitude': 34.102349, 'longitude': -118.344672}
{'myspaceid': 'WU759', 'latitude': 34.051792, 'longitude': -118.27393}
{'myspaceid': 'CB118', 'latitude': 34.039121, 'longitude': -118.266015}
{'myspaceid': 'WF60', 'latitude': 34.072672, 'longitude': -118.369344}
{'myspaceid': 'WS490', 'latitude': 34.04847, 'longitude': -118.445724}
{'myspaceid': 'WU267', 'latitude': 34.052744, 'longitude': -118.262942}
{'myspaceid': 'OS521', 'latitude': 34.031765, 'longitude': -118.456463}
{'myspaceid': 'VN1001', 'latitude': 34.187711, 'longitude': -118.447842}
{'myspaceid': 'VW164', 'latitude': 34.057641, 'longitude': -118.29

{'myspaceid': 'UC213', 'latitude': 34.136285, 'longitude': -118.362261}
{'myspaceid': 'MV107', 'latitude': 34.002644, 'longitude': -118.435998}
{'myspaceid': 'VN668', 'latitude': 34.183911, 'longitude': -118.444284}
{'myspaceid': 'SV165', 'latitude': 34.098084, 'longitude': -118.334757}
{'myspaceid': 'WW582', 'latitude': 34.060423, 'longitude': -118.302065}
{'myspaceid': 'PB72', 'latitude': 34.04821, 'longitude': -118.350954}
{'myspaceid': 'SC255', 'latitude': 34.143058, 'longitude': -118.391842}
{'myspaceid': 'LC63', 'latitude': 34.088744, 'longitude': -118.376097}
{'myspaceid': 'PB315', 'latitude': 34.048234, 'longitude': -118.355913}
{'myspaceid': 'LM677', 'latitude': 34.070086, 'longitude': -118.343893}
{'myspaceid': 'UC49', 'latitude': 34.134105, 'longitude': -118.358855}
{'myspaceid': 'CC1', 'latitude': 34.050713, 'longitude': -118.240466}
{'myspaceid': 'OS132', 'latitude': 34.036002, 'longitude': -118.443547}
{'myspaceid': 'MD422', 'latitude': 34.064111, 'longitude': -118.206364

{'myspaceid': '61141', 'latitude': 33.996416, 'longitude': -118.477276}
{'myspaceid': '61365', 'latitude': 33.987328, 'longitude': -118.47298}
{'myspaceid': '56245', 'latitude': 34.048119, 'longitude': -118.442176}
{'myspaceid': '56163', 'latitude': 34.054994, 'longitude': -118.428048}
{'myspaceid': '52208', 'latitude': 34.025399, 'longitude': -118.290345}
{'myspaceid': '61242', 'latitude': 33.996253, 'longitude': -118.477382}
{'myspaceid': '93118', 'latitude': 34.0894, 'longitude': -118.2766}
{'myspaceid': '56418', 'latitude': 34.049179, 'longitude': -118.438076}
{'myspaceid': '93236', 'latitude': 34.090552, 'longitude': -118.278109}
{'myspaceid': 'HP273', 'latitude': 34.106538, 'longitude': -118.199787}
{'myspaceid': '66414', 'latitude': 34.101496, 'longitude': -118.332776}
{'myspaceid': '56850', 'latitude': 34.057881, 'longitude': -118.422996}
{'myspaceid': '56463', 'latitude': 34.050264, 'longitude': -118.434252}
{'myspaceid': '82712', 'latitude': 33.979522, 'longitude': -118.46617

{'myspaceid': 'L72', 'latitude': 34.073552, 'longitude': -118.323754}
{'myspaceid': 'L96', 'latitude': 34.073832, 'longitude': -118.323515}
{'myspaceid': 'L66', 'latitude': 34.073743, 'longitude': -118.323754}
{'myspaceid': 'L24', 'latitude': 34.075503, 'longitude': -118.32376}
{'myspaceid': 'ED619B', 'latitude': 34.034412, 'longitude': -118.250222}
{'myspaceid': 'C1226', 'latitude': 34.026892, 'longitude': -118.282505}
{'myspaceid': 'CT65', 'latitude': 34.060077, 'longitude': -118.238371}
{'myspaceid': 'CC387', 'latitude': 34.052277, 'longitude': -118.250108}
{'myspaceid': 'CC855', 'latitude': 34.053336, 'longitude': -118.252766}
{'myspaceid': 'L107', 'latitude': 34.074175, 'longitude': -118.323516}
{'myspaceid': 'L42', 'latitude': 34.074579, 'longitude': -118.32376}
{'myspaceid': 'L86', 'latitude': 34.073186, 'longitude': -118.323512}
{'myspaceid': 'CC23', 'latitude': 34.051436, 'longitude': -118.238634}
{'myspaceid': 'L26', 'latitude': 34.075446, 'longitude': -118.32376}
{'myspaceid

{'myspaceid': 'VN1033', 'latitude': 34.188748, 'longitude': -118.447211}
{'myspaceid': 'TZ345', 'latitude': 34.172412, 'longitude': -118.557904}
{'myspaceid': 'AE93', 'latitude': 34.046735, 'longitude': -118.237709}
{'myspaceid': 'WA1071', 'latitude': 34.055164, 'longitude': -118.27375}
{'myspaceid': 'WS246', 'latitude': 34.047952, 'longitude': -118.435735}
{'myspaceid': 'BF171A', 'latitude': 34.08361, 'longitude': -118.355829}
{'myspaceid': 'R20', 'latitude': 34.201234, 'longitude': -118.533}
{'myspaceid': 'WH392', 'latitude': 34.164124, 'longitude': -118.628485}
{'myspaceid': 'AE5', 'latitude': 34.044837, 'longitude': -118.237212}
{'myspaceid': 'C1188', 'latitude': 34.025506, 'longitude': -118.286651}
{'myspaceid': 'BF761', 'latitude': 34.0809, 'longitude': -118.361355}
{'myspaceid': 'SVV492', 'latitude': 34.089375, 'longitude': -118.294212}
{'myspaceid': 'VN647', 'latitude': 34.182347, 'longitude': -118.446466}
{'myspaceid': 'SC415', 'latitude': 34.141205, 'longitude': -118.37246}
{

...

{'myspaceid': 'VW1839', 'latitude': 34.047526, 'longitude': -118.290788}
{'myspaceid': 'WU789', 'latitude': 34.050242, 'longitude': -118.272244}
{'myspaceid': 'SO224', 'latitude': 34.153366, 'longitude': -118.459549}
{'myspaceid': 'TZ444', 'latitude': 34.172317, 'longitude': -118.545566}
{'myspaceid': 'P29', 'latitude': 34.047051, 'longitude': -118.52618}
{'myspaceid': 'CB2327', 'latitude': 34.032931, 'longitude': -118.254419}
{'myspaceid': 'PR532', 'latitude': 34.054224, 'longitude': -118.376031}
{'myspaceid': 'SV362', 'latitude': 34.09731, 'longitude': -118.330358}
{'myspaceid': 'PR811', 'latitude': 34.050608, 'longitude': -118.384796}
{'myspaceid': 'EN90', 'latitude': 34.156475, 'longitude': -118.48151}
{'myspaceid': 'SA231', 'latitude': 34.076049, 'longitude': -118.255418}
{'myspaceid': 'VW363', 'latitude': 34.063172, 'longitude': -118.295296}
{'myspaceid': 'SL88', 'latitude': 34.100455, 'longitude': -118.273179}
{'myspaceid': 'WV506', 'latitude': 34.063782, 'longitude': -118.44458

{'myspaceid': 'WA1253', 'latitude': 34.064027, 'longitude': -118.27188}
{'myspaceid': 'CB1749', 'latitude': 34.037121, 'longitude': -118.25851}
{'myspaceid': 'ED85', 'latitude': 34.035896, 'longitude': -118.245893}
{'myspaceid': 'TZ414', 'latitude': 34.172439, 'longitude': -118.549063}
{'myspaceid': 'SS121', 'latitude': 34.046265, 'longitude': -118.44879}
{'myspaceid': 'SC324', 'latitude': 34.140891, 'longitude': -118.386886}
{'myspaceid': 'SV780', 'latitude': 34.093699, 'longitude': -118.338742}
{'myspaceid': 'SV315', 'latitude': 34.097901, 'longitude': -118.325862}
{'myspaceid': 'WU635', 'latitude': 34.051982, 'longitude': -118.267115}
{'myspaceid': 'NH998', 'latitude': 34.152308, 'longitude': -118.362796}
{'myspaceid': 'VW1112', 'latitude': 34.06221, 'longitude': -118.282845}
{'myspaceid': 'WA649', 'latitude': 34.061536, 'longitude': -118.280668}
{'myspaceid': 'AE373', 'latitude': 34.049905, 'longitude': -118.233547}
{'myspaceid': 'SV609', 'latitude': 34.098262, 'longitude': -118.33

{'myspaceid': 'WH678', 'latitude': 34.166576, 'longitude': -118.591227}
{'myspaceid': 'OS193', 'latitude': 34.036536, 'longitude': -118.440388}
{'myspaceid': 'SL51', 'latitude': 34.105186, 'longitude': -118.273047}
{'myspaceid': 'HP59', 'latitude': 34.109765, 'longitude': -118.192212}
{'myspaceid': 'EN228', 'latitude': 34.159717, 'longitude': -118.49849}
{'myspaceid': 'PW355', 'latitude': 34.037662, 'longitude': -118.434354}
{'myspaceid': 'R383', 'latitude': 34.197284, 'longitude': -118.536166}
{'myspaceid': 'ER142', 'latitude': 34.139086, 'longitude': -118.210371}
{'myspaceid': 'CC1134', 'latitude': 34.059459, 'longitude': -118.242127}
{'myspaceid': 'WV155A', 'latitude': 34.063572, 'longitude': -118.447352}
{'myspaceid': 'P243', 'latitude': 34.046023, 'longitude': -118.526223}
{'myspaceid': 'SL428', 'latitude': 34.081671, 'longitude': -118.271762}
{'myspaceid': 'SV328', 'latitude': 34.097906, 'longitude': -118.32299}
{'myspaceid': 'VN140', 'latitude': 34.189635, 'longitude': -118.4500

{'myspaceid': 'PR721', 'latitude': 34.053688, 'longitude': -118.374977}
{'myspaceid': 'WA876', 'latitude': 34.058694, 'longitude': -118.280008}
{'myspaceid': 'NH610', 'latitude': 34.158556, 'longitude': -118.370931}
{'myspaceid': 'CB1591', 'latitude': 34.036237, 'longitude': -118.25595}
{'myspaceid': 'SL602', 'latitude': 34.082303, 'longitude': -118.260256}
{'myspaceid': 'WP89', 'latitude': 33.980308, 'longitude': -118.462701}
{'myspaceid': 'CB4105', 'latitude': 34.032516, 'longitude': -118.262638}
{'myspaceid': 'SV608A', 'latitude': 34.098428, 'longitude': -118.336379}
{'myspaceid': 'CB1854', 'latitude': 34.045892, 'longitude': -118.246021}
{'myspaceid': 'WH163', 'latitude': 34.167769, 'longitude': -118.596372}
{'myspaceid': 'ED402', 'latitude': 34.031699, 'longitude': -118.251689}
{'myspaceid': 'CB3851', 'latitude': 34.037024, 'longitude': -118.265215}
{'myspaceid': 'YO67', 'latitude': 34.121231, 'longitude': -118.206029}
{'myspaceid': 'CT311', 'latitude': 34.065401, 'longitude': -11

{'myspaceid': 'PR374', 'latitude': 34.056786, 'longitude': -118.396607}
{'myspaceid': 'SO907', 'latitude': 34.14784, 'longitude': -118.427891}
{'myspaceid': 'CB4248', 'latitude': 34.043488, 'longitude': -118.261168}
{'myspaceid': 'SV291B', 'latitude': 34.096773, 'longitude': -118.32889}
{'myspaceid': 'WH440', 'latitude': 34.166267, 'longitude': -118.622105}
{'myspaceid': 'HO958A', 'latitude': 34.103015, 'longitude': -118.333267}
{'myspaceid': 'SP522', 'latitude': 33.737913, 'longitude': -118.285211}
{'myspaceid': 'CP150', 'latitude': 34.200583, 'longitude': -118.601673}
{'myspaceid': 'CT299', 'latitude': 34.066378, 'longitude': -118.237841}
{'myspaceid': 'MM123', 'latitude': 34.06164, 'longitude': -118.345289}
{'myspaceid': 'WF434', 'latitude': 34.06539, 'longitude': -118.372585}


In [37]:

data_list = []
for x in range(0, 4):
    filename = f"page_{x}.json"
    with open(filename) as json_file:  
        data = json.load(json_file)
        for s in data:
            data_list.append(s)

transformed_df1 = pd.DataFrame(data_list)
transformed_df1

,:@computed_region_2dna_qi2s,:@computed_region_k96s_3jcv,:@computed_region_kqwf_mjcx,:@computed_region_qz3q_ghft,:@computed_region_tatf_ua23,:@computed_region_ur2y_g4cx,blockface,latlng,metertype,parkingpolicy,raterange,ratetype,spaceid,stcl,timelimit
0,58,430,8,23445,456,16,4900 SUNSET BL,"{'latitude': '34.098024', 'longitude': '-118.2...",Single-Space,"TANS 4P-7P Mon-Fri | PKG 8A-4P Mon-Fri, 8A-8P Sat",$1.00,FLAT,VS336,NaN,2HR
1,76,703,9,23082,1018,NaN,700 E PICO BLVD,"{'latitude': '34.032054', 'longitude': '-118.2...",Single-Space,8A-6P Mon-Sat,$0.5 - $1,TOD,ED396,NaN,2HR
2,76,693,9,23076,1291,NaN,1300 S GRAND AVE,"{'latitude': '34.038125', 'longitude': '-118.2...",Single-Space,8A-8P Mon-Sat,$0.50,FLAT,CB211A,NaN,4HR
3,90,473,8,23444,1297,NaN,3100 SUNSET BL,"{'latitude': '34.084343', 'longitude': '-118.2...",Single-Space,8A-8P Mon-Sat,$1.00,FLAT,SL408,NaN,2HR
4,18,229,3,19733,288,NaN,7001 VAN NUYS BL,"{'latitude': '34.198035', 'longitude': '-118.4...",Single-Space,8A-8P Mon-Sat,$1.00,FLAT,VN1,NaN,2HR
5,82,414,8,23446,1303,NaN,1700 EL CERRITO PL,"{'latitude': '34.102349', 'longitude': '-118.3...",Single-Space,"TANS 8A-5A Fri | PKG 8A-8P Mon-Thu, 8A-12A Fri...",$1.5 - $3,TOD,HO1021A,12P-3P Mon,2HR
6,68,571,11,23078,628,NaN,800 BEACON AV,"{'latitude': '34.051792', 'longitude': '-118.2...",Single-Space,8A-6P Mon-Sat,$1.00,FLAT,WU759,12P-3P Thu,2HR
7,76,693,9,23076,1291,NaN,1301 S HOPE ST,"{'latitude': '34.039121', 'longitude': '-118.2...",Single-Space,8A-8P Mon-Sat,$1 - $2,TOD,CB118,NaN,4HR
8,26,629,6,23679,1095,NaN,8201 3RD ST,"{'latitude': '34.072672', 'longitude': '-118.3...",Single-Space,"8A-8P Mon-Sat, 11A-8P Sun",$1.00,FLAT,WF60,NaN,2HR
9,72,855,6,23443,748,9,1600 PONTIUS AV,"{'latitude': '34.04847', 'longitude': '-118.44...",Single-Space,8A-8P Mon-Sat,$1.00,FLAT,WS490,NaN,1HR


In [38]:
transformed_df1.rename(columns = 
                                   {"blockface": "address", 
                                    "metertype": "meter_type", 
                                    "parkingpolicy": "parking_policy", 
                                    "raterange": "rate_range", 
                                    "ratetype": "rate_type", 
                                    "spaceid": "id", 
                                    "stcl": "street_cleaning", 
                                    "timelimit": "parking_limit",}, 
                                   inplace=True)

In [39]:
transformed_df1.drop([':@computed_region_2dna_qi2s',
                      ':@computed_region_k96s_3jcv',
                      ':@computed_region_kqwf_mjcx',
                            ':@computed_region_qz3q_ghft',
                            ':@computed_region_tatf_ua23', 
                            ':@computed_region_ur2y_g4cx',
                      "latlng"
                     ], 
                           axis = 1, inplace = True)

In [45]:
la_meter_inventory_update_1 = pd.DataFrame(transformed_df1)
la_meter_inventory_update_1

,address,meter_type,parking_policy,rate_range,rate_type,id,street_cleaning,parking_limit
0,4900 SUNSET BL,Single-Space,"TANS 4P-7P Mon-Fri | PKG 8A-4P Mon-Fri, 8A-8P Sat",$1.00,FLAT,VS336,NaN,2HR
1,700 E PICO BLVD,Single-Space,8A-6P Mon-Sat,$0.5 - $1,TOD,ED396,NaN,2HR
2,1300 S GRAND AVE,Single-Space,8A-8P Mon-Sat,$0.50,FLAT,CB211A,NaN,4HR
3,3100 SUNSET BL,Single-Space,8A-8P Mon-Sat,$1.00,FLAT,SL408,NaN,2HR
4,7001 VAN NUYS BL,Single-Space,8A-8P Mon-Sat,$1.00,FLAT,VN1,NaN,2HR
5,1700 EL CERRITO PL,Single-Space,"TANS 8A-5A Fri | PKG 8A-8P Mon-Thu, 8A-12A Fri...",$1.5 - $3,TOD,HO1021A,12P-3P Mon,2HR
6,800 BEACON AV,Single-Space,8A-6P Mon-Sat,$1.00,FLAT,WU759,12P-3P Thu,2HR
7,1301 S HOPE ST,Single-Space,8A-8P Mon-Sat,$1 - $2,TOD,CB118,NaN,4HR
8,8201 3RD ST,Single-Space,"8A-8P Mon-Sat, 11A-8P Sun",$1.00,FLAT,WF60,NaN,2HR
9,1600 PONTIUS AV,Single-Space,8A-8P Mon-Sat,$1.00,FLAT,WS490,NaN,1HR


In [42]:
# Create database connection
connection_string = "root:root@127.0.0.1:3306/parking_db"
engine = create_engine(f'mysql+pymysql://{connection_string}')

In [43]:
# Confirm tables
engine.table_names()

['meters_inventory', 'meters_inventory_full']

In [46]:
# Load DataFrames into database
la_meter_inventory_update_1.to_sql(name = 'meters_inventory_full', con = engine, if_exists = 'append', index = False) 